In [1]:
import os
import urllib
# import webapp2
import jinja2
import pandas as pd
import json


from apiclient.discovery import build
from optparse import OptionParser

from youtube_transcript_api import YouTubeTranscriptApi

In [2]:
with open("youtube_stats.json", 'r') as f:
    data = json.load(f)

In [4]:
data.keys()

dict_keys(['tags', 'channelId', 'channelTitle', 'categoryId', 'title', 'videoId', 'viewCount', 'likeCount', 'dislikeCount', 'commentCount', 'favoriteCount', 'description'])

In [6]:
all_transcripts = []

id_list = data['videoId']

for i, video_id in enumerate(data['videoId']):
    
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    except:
        transtript_list = None
        pass
    
    all_transcripts.append(transcript_list)
    if not transcript_list:
        print("MISSING TRANSCRIPT:", data["title"][i])
    
    if i % 100 == 0:
        print("PROCESSED: ",i)

PROCESSED:  0
PROCESSED:  100
PROCESSED:  200
PROCESSED:  300
PROCESSED:  400
PROCESSED:  500
PROCESSED:  600
PROCESSED:  700
PROCESSED:  800
PROCESSED:  900
PROCESSED:  1000
PROCESSED:  1100
PROCESSED:  1200
PROCESSED:  1300
PROCESSED:  1400
PROCESSED:  1500
PROCESSED:  1600
PROCESSED:  1700
PROCESSED:  1800
PROCESSED:  1900


In [20]:
data['transcript_api'] = all_transcripts

In [21]:
data['transcript_api'][0].find_transcript(['en']).fetch()

[{'text': '[Music]', 'start': 1.98, 'duration': 4.88},
 {'text': 'most men live lives of quiet desperation',
  'start': 4.68,
  'duration': 4.2},
 {'text': "it's one of my favorite quotes ever",
  'start': 6.86,
  'duration': 3.15},
 {'text': "because it's true", 'start': 8.88, 'duration': 6.73},
 {'text': '[Music]', 'start': 10.01, 'duration': 8.91},
 {'text': "you're just in this world where you just",
  'start': 15.61,
  'duration': 8.23},
 {'text': "can't wait to just run away but I think",
  'start': 18.92,
  'duration': 6.15},
 {'text': 'one of the reasons why these people have',
  'start': 23.84,
  'duration': 3.0},
 {'text': 'this deep-seated anger and resentment is',
  'start': 25.07,
  'duration': 3.72},
 {'text': "there's a bunch of people out there that",
  'start': 26.84,
  'duration': 4.92},
 {'text': 'have these lives that are deeply',
  'start': 28.79,
  'duration': 5.55},
 {'text': 'unsatisfying because I think there are',
  'start': 31.76,
  'duration': 5.25},
 {'text

In [24]:

#REDO WITH ID AND TRANSCRIPT API IN ONE PLACE

transcript_dict_all = {}
transcript_dict_man = {}

for i, video_id in enumerate(data['videoId']):
    
    transcript_api = data['transcript_api'][i]
    try:
        en_transcript = transcript_api.find_transcript(['en']).fetch()
        transcript_dict_all[video_id] = en_transcript
    except:
        pass
    
    #try to find a manual transcript if it exists in transcript_list - not often
    try:
        en_transcript_manual = transcript_api.find_manually_created_transcript(['en']).fetch()
        transcript_dict_man[video_id] = en_transcript_manual
    except:
        pass
    
    if i % 200 == 0:
        print("PROCESSED: ", i)
        print("---")

PROCESSED:  0
---
PROCESSED:  200
---
PROCESSED:  400
---
PROCESSED:  600
---
PROCESSED:  800
---
PROCESSED:  1000
---
PROCESSED:  1200
---
PROCESSED:  1400
---
PROCESSED:  1600
---
PROCESSED:  1800
---


In [31]:
print(transcript_dict_man.values())

dict_values([[{'text': 'oh hi! oh yes... i did... i quit. Oh! you too?\xa0\nmaybe? So, I quit. I was a mid tier lawyer\xa0\xa0', 'start': 0.88, 'duration': 10.72}, {'text': 'in a Big 4 law firm in Singapore. For over\xa0\n6 years I slogged my guts out. The job paid\xa0\xa0', 'start': 11.6, 'duration': 6.72}, {'text': 'me a nice 6 figure salary. When i talked about my\xa0\njob it always sounds, like, so glamourous. In the\xa0\xa0', 'start': 18.32, 'duration': 7.04}, {'text': 'eyes of my parents, the people around me, I had\xa0\n“made it”. I was “doing well”. Except, I wasn’t,\xa0\xa0', 'start': 25.36, 'duration': 8.4}, {'text': 'not really. Really, I was just going through the\xa0\nmotions, day after day, and feeling so tired and\xa0\xa0', 'start': 34.56, 'duration': 8.16}, {'text': 'so … alone. The colleagues working alongside me\xa0\nall seemed happy enough. In fact most of them were\xa0\xa0', 'start': 43.6, 'duration': 7.12}, {'text': 'actively buying into the whole thing, even when 

In [39]:
def process_transcripts(raw_dict_in, outfile='processed_transcripts.json'):
    
    """Writes a json file from the raw transcript dict returned from scrape_transcripts()
    
    Returns processed_transcripts with video_ids as keys and the following items:
        list: list of sentence strings from 'text' field of raw_dict
        str: combined string from 'text' field of raw_dict
    
    """
    transcript_out = "processed_transcripts.json"

    processed_transcripts = {}
    for video_id, transcript in raw_dict_in.items():
#         print(video_id)
    #     print(transcript)
        processed_transcripts[video_id] = {
            "list": [],
            "str": ""
        }
        transcript_list = []
        transcript_str = ""
        for item in transcript:
            text_line = item['text']
            transcript_list.append(text_line)
            transcript_str += text_line 
            transcript_str += " "
    #     print(transcript_list)
    #     print(transcript_str)
        processed_transcripts[video_id]['list'] = transcript_list
        processed_transcripts[video_id]['str'] = transcript_str

#     print(processed_transcripts)

    with open(outfile, 'w', encoding='utf-8') as f:
        json.dump(processed_transcripts, f, ensure_ascii=False, indent=4)

In [40]:
process_transcripts(transcript_dict_man, "processed_MAN.json")
process_transcripts(transcript_dict_all, "processed_ALL.json")